Python 3.11.2에 연결됨

In [1]:
import requests
import json
import math
import time

HOST = "http://127.0.0.1:56123"

REGISTRATION_URL = f"{HOST}/api/v4/rest-auth/registration/"
LOGIN_URL = f"{HOST}/api/v4/rest-auth/login/"
LOGOUT_URL = f"{HOST}/api/v4/rest-auth/logout/"
UPLOAD_URL = f"{HOST}/api/v4/upload"
DIVIDE_URL = f"{HOST}/api/v4/parser/divide"
PARSE_URL = f"{HOST}/api/v4/parser/parse"
ANNOTATE_URL = f"{HOST}/api/v4/annotator/annotate"
CORPUSES_URL = f"{HOST}/api/v4/corpuses/"

url = "http://127.0.0.1:56123"


In [2]:
join_membership = {
    "username": "abc111",
    "email": "abcdefghi12@naver.com",
    "password1": "testaaaaaa!",
    "password2": "testaaaaaa!"
}

join_membership_response = requests.post(REGISTRATION_URL, data=join_membership)

login = {
    "username": "abc111",
    "email": "abcdefghi12@naver.com",
    "password": "testaaaaaa!"
}

login_response= requests.post(LOGIN_URL, json=login)
login_data = login_response.json()
token = login_data.get('key')

login_key = {
    'Authorization': f'Token {token}'
}


In [3]:
texts = [
    {"original_text": "Die Alpen erstrecken sich über mehrere Länder Europas ..."},
    {"original_text": "Der Rhein ist einer der längsten Flüsse Europas ..."},
    {"original_text": "München, bekannt für das Oktoberfest ..."},
    {"original_text": "Der Schwarzwald ist ein großes Waldgebiet ..."},
    {"original_text": "Die Insel Rügen ist die größte Insel Deutschlands ..."},
    {"original_text": "Die Wartburg, eine mittelalterliche Burg in Thüringen ..."}
]


In [4]:
time_list = []
annotate_list = []
annotators = ["chatgpt_ft0", "dummy"]

for annotator_name in annotators:
    for text in texts:
        upload_response = requests.post(UPLOAD_URL, json=text, headers=login_key)
        corpus_id = upload_response.json().get('corpus_id')

        divide_data = {
            "corpus_id": corpus_id,
            "divide_options": json.dumps({"p_delims": ["\n"]})
        }
        divide_response = requests.post(DIVIDE_URL, json=divide_data, headers=login_key)

        parse_data = {
            "corpus_id": corpus_id,
            "parse_options": json.dumps({"t_delims": " \t\n\r\u000b\f"})
        }
        parse_response = requests.post(PARSE_URL, json=parse_data, headers=login_key)

        start = time.time()

        annotate_data = {
            "corpus_id": corpus_id,
            "annotate_options": json.dumps({
                "lang_from": "German",
                "lang_to": "English",
                "annotator_name": annotator_name
            })
        }
        annotate_response = requests.post(ANNOTATE_URL, json=annotate_data, headers=login_key)
        
        end = time.time()
        print(annotator_name)
        print(f"{end - start:.5f} sec")
        time_list.append(end - start)
        annotate_list.append(annotator_name)


chatgpt_ft0
2.73698 sec
chatgpt_ft0
1.26771 sec
chatgpt_ft0
1.07898 sec
chatgpt_ft0
1.35017 sec
chatgpt_ft0
1.44054 sec
chatgpt_ft0
1.24414 sec
dummy
0.12476 sec
dummy
0.15148 sec
dummy
0.15495 sec
dummy
0.26533 sec
dummy
0.17129 sec
dummy
0.20500 sec


In [5]:
corpuses_respons = requests.get(CORPUSES_URL, headers=login_key)
corpuse_history = corpuses_respons.json()

start = len(texts)
num = int(corpuse_history[-1].get('corpus_id'))  # 마지막 코퍼스의 ID 가져오기

for number in range(num):
    Unknown_count = 0
    token_sum = 0
    for corpus_record in corpuse_history[number]['corpuses_history']:
        for paragraph in corpus_record['paragraphs']:
            if paragraph['pstate'] == "ANNOTATED":
                for token in paragraph.get('tokens', []):
                    if not token['is_delimiter']:
                        token_sum += 1
                        if token.get('gloss') == "!UNKNOWN":
                            Unknown_count += 1
                            print(token)

    if token_sum > 0:
        Return_rate = ((token_sum - Unknown_count) / token_sum) * 100
        print(number, "annotator_name:", annotate_list[number], "time:", round(float(time_list[number]), 2), ", return_rate:", round(Return_rate, 2))
    else:
        print(number, "time:", round(float(time_list[number]), 2), ", No tokens to calculate return rate")


0 annotator_name: chatgpt_ft0 time: 2.74 , return_rate: 100.0
1 annotator_name: chatgpt_ft0 time: 1.27 , return_rate: 100.0
2 annotator_name: chatgpt_ft0 time: 1.08 , return_rate: 100.0
3 annotator_name: chatgpt_ft0 time: 1.35 , return_rate: 100.0
4 annotator_name: chatgpt_ft0 time: 1.44 , return_rate: 100.0
5 annotator_name: chatgpt_ft0 time: 1.24 , return_rate: 100.0
6 annotator_name: dummy time: 0.12 , return_rate: 100.0
7 annotator_name: dummy time: 0.15 , return_rate: 100.0
8 annotator_name: dummy time: 0.15 , return_rate: 100.0
9 annotator_name: dummy time: 0.27 , return_rate: 100.0
10 annotator_name: dummy time: 0.17 , return_rate: 100.0
11 annotator_name: dummy time: 0.2 , return_rate: 100.0
